In [13]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from nltk.tokenize import sent_tokenize
from collections import Counter
import numpy as np


dvach = open('2ch_corpus.txt', encoding='utf-8').read()[:50000]
lenta = open('lenta.txt', encoding='utf-8').read()[:50000]


def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams


def generate(matrix, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    current_bigram = start
    for i in range(n):
        chosenindex = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        appended = id2word[chosenindex]
        text.append(appended)
        if appended == '<end>':
            current_idx = bigram2id['<start> <start>']
            current_bigram = '<start> <start>'
            continue
        current_bigram = current_bigram.split()[1]+' '+appended
        current_idx = bigram2id[current_bigram]
    return ' '.join(text)



sentences_dvach = [['<start>']+['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
#
sentences_lenta = [['<start>']+['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(lenta)]


unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()
#
unigrams_lenta = Counter()
bigrams_lenta = Counter()
trigrams_lenta = Counter()


for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))
#
for sentence in sentences_lenta:
    unigrams_lenta.update(sentence)
    bigrams_lenta.update(ngrammer(sentence))
    trigrams_lenta.update(ngrammer(sentence, n=3))
    

matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}
#
matrix_lenta = np.zeros((len(bigrams_lenta), 
                   len(unigrams_lenta)))
id2word_lenta = list(unigrams_lenta)
word2id_lenta = {word:i for i, word in enumerate(id2word_lenta)}
id2bigram_lenta = list(bigrams_lenta)
bigram2id_lenta = {word:i for i, word in enumerate(id2bigram_lenta)}


for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    matrix_dvach[bigram2id_dvach[word1+' '+word2]][word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                             bigrams_dvach[word1+' '+word2])
#
for ngram in trigrams_lenta:
    word1, word2, word3 = ngram.split()
    matrix_lenta[bigram2id_lenta[word1+' '+word2]][word2id_lenta[word3]] =  (trigrams_lenta[ngram]/
                                                                             bigrams_lenta[word1+' '+word2])
    
    
dvachfashion = generate(matrix_dvach, id2word_dvach, bigram2id_dvach).replace('<end>', '\n')
lentafashion = generate(matrix_lenta, id2word_lenta, bigram2id_lenta).replace('<end>', '\n')

with open('results.txt', 'w', encoding='utf-8') as f:
    f.write(dvachfashion)
    f.write('\n\n')
    f.write(lentafashion)
